<a href="https://colab.research.google.com/github/paulopatto/AI_for_Resumes/blob/master/BRB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Be Right Back (BRB)

Criando uma versão virtual e análise sobre personalidade com base em dados de redes sociais.

## Introdução: Revivendo Conversas com Agentes LLM Inspirados em "Be Right Back"

O luto é uma experiência universal e complexa, marcada pela dor da perda e pelo desejo de manter a conexão com aqueles que se foram. O episódio "[Be Right Back](https://en.wikipedia.org/wiki/Be_Right_Back) (S2E1)" da série Black Mirror explora essa temática de forma inquietante e reflexiva, apresentando um futuro onde a tecnologia permite a criação de replicas virtuais de pessoas falecidas a partir de suas mídias sociais e dados pessoais.

Inspirados por essa ficção científica, embarcamos em um projeto ambicioso: a criação de um sistema de agentes LLM (Large Language Models) capaz de imitar padrões conversacionais de indivíduos com base em seus históricos de interação em plataformas online, como WhatsApp e redes sociais. Através da integração de modelos de linguagem avançados como Gemini Pro, CrewAI e Longchain, nosso objetivo é desenvolver um sistema que permita:

- **Recriar conversas ou simular conversas**: O sistema será capaz de analisar logs de conversas do WhatsApp ~~e outros canais online~~(melhoria futura) para capturar o estilo, vocabulário e tom de voz do usuário, permitindo a geração de novas conversas que simulam a sua presença.
- **Manter a conexão**: Ao oferecer a possibilidade de interagir com uma versão virtual  da pessoa, no caso proposto na série, com uma versão virtual do falecido, o sistema visa auxiliar no processo de luto, fornecendo um espaço para compartilhar memórias, expressar sentimentos e manter viva a lembrança da pessoa amada. Porém este projeto entende que isso pode trazer um prolongamento do sofrimento do luto e pode ser usado em situções onde uma pessoa VIVA possa quere colocar uma versão virtual de si para responder mensganes e fazer tarefas do dia-a-dia.

### Considerações Éticas

O desenvolvimento de um sistema como este levanta importantes questões éticas que precisam ser cuidadosamente ponderadas. É fundamental garantir que a tecnologia seja utilizada de forma responsável e respeitosa, evitando a manipulação emocional dos usuários e a perpetuação de crenças falsas sobre a consciência ou imortalidade.
Outras obras do cinema como [Transcendente - A revolução](https://en.wikipedia.org/wiki/Transcendence_(2014_film))

## Gerando dados para treinamento dos agentes

Para que nossos agentes (crew) possam aprender como você (ou a pessoa que você quer análisar) falam, vamos precisar pegar um padrão conversacional e para  imitar e/ou gerar uma linguagem humana de forma natural e envolvente e para que nossos agentes alcancem o máximo potencial, é crucial treinalo com dados de  alta qualidade e onde a pessoal se mostrou de forma mais natural para o propósito da aplicação. É aí que entra a riqueza de informações contida nas conversas do aplicativo WhatsApp (mas relaxa em nossa versão Google Colab, você vai fazer isso dentro do seu Google Drive e você só vai compartilhar suas informações com o Gemini do Google).


### Exportando Conversas do WhatsApp (dump)

#### Conversas especificas

Poder ser que vocẽ queira compartilhar apenas alguns contextos e por isso pode escolher as conversas que vai exportar, para isso tanto em Android como iPhone seiga esses passos:

- Abra o WhatsApp e acesse a conversa desejada.
- Toque nos três pontos no canto superior direito da tela.
- Selecione "Mais" > "Exportar conversa".
- Escolha entre "Sem arquivos" ou "Incluir arquivos de mídia".
- Confirme a exportação e selecione o local para salvar o arquivo (TXT).

#### Utilizando Backups Existentes

Essa opção vai pegar todo o universo conversacional do Whatsapp nos mais diversos contextos, pode ser mais arriscado mas é a mais rica, lembra na série quando a Martha compsrtilha todo o arquivo de e-mails do Ash em como ele fica bem melhor? Pois é.

**Na plataforma Google Android você pode seguir:**

- Localize o arquivo de backup do WhatsApp (geralmente em `Armazenamento Interno > WhatsApp > Databases`).
- Extraia o arquivo de backup usando um software de extração.
- Acesse a pasta "msgstore.db" dentro do backup extraído.
- Utilize ferramentas específicas (como o SQLite Viewer) para abrir e exportar as conversas desejadas.

**Na plataforma Apple iPhone**

- Conecte o iPhone ao computador e abra o iTunes.
- Selecione o dispositivo e clique em "Backup Agora".
- Localize o backup do WhatsApp no computador (geralmente em `Usuários > ${Nome do Usuário} > AppData > Roaming > Apple Computer > MobileSync > Backup`).
- Extraia o arquivo de backup usando um software de extração.
- Acesse a pasta "ChatDatabase.sqlite" dentro do backup extraído.
- Utilize ferramentas específicas (como o SQLite Viewer) para abrir e exportar as conversas desejadas.

(*) Sei que ficou bem nerd e isso pode melhor muito no processo


### Organizando para subir

Coloque todas as conversas (*.txt) exportadas pelo dump em uma mesma pasta, não vamos compactar pois em alguns testes a compactação não ficou legal, mas isso pode ser corrigido no futuro.

**Versão Google Colab**

Na versão Google colab você pode subir para a pasta de arquivos temposários com o nome da pasta como `Conversas`

**Versão StandAlone python**

Ainda não sei como seria a melhor forma!


Feito isso vamos rodar o modelo!

## Desenvolvimento

O desenvolviment

### Setup Env

- Install Libs
- Setup APIKEY
- Setup Model com Longchain

In [2]:
!pip install -q -U google-generativeai crewai langchain_google_genai crewai[tools]

In [3]:
from google.colab import userdata
import google.generativeai as genai

LLM_MODEL = 'gemini-pro'
GEMINI_API_KEY=userdata.get('GEMINI_API_KEY')
genai.configure(api_key=GEMINI_API_KEY)
model = genai.GenerativeModel(LLM_MODEL)
#response = model.generate_content("Write a story about a magic backpack.")
#print(response.text)


In [4]:
import os
from langchain_google_genai import ChatGoogleGenerativeAI
from crewai import Agent, Task, Crew, Process

llm = ChatGoogleGenerativeAI(
  model=LLM_MODEL,
  verbose = True,
  temperature = 0.6,
  google_api_key=GEMINI_API_KEY
)

# Check if chats_folder exists, if no creates it
chats_folder = '/content/Conversas'
if not os.path.isdir(chats_folder):
  os.mkdir(chats_folder)

### Agentes

Criação dos agentes

#### Profilers



In [11]:
#
from crewai_tools import DirectoryReadTool

readDirectoryTool = DirectoryReadTool(directory=chats_folder)
profiler_agent = Agent(
  role='Analista de perfil',
  goal=f"""
  Dado um conjunto de mensagens de texto
  E dado um determinado Ator
  Ler mensagens as de texto
  Sua expertise consiste em identificar padrões de mensagens nos chats e classificá-los nos mais diversos contextos.
  Irá fazer uma análise do padrão de conversação do Ator desejado nos diferentes contextos em que se encontra
  """,
  backstory="""
  Você trabalha em um departamento de análise de mensagen, analisando e classificando textos de mensagens.
  Sua expertise consiste em identificar padrões de mensagens e classificá-los nos mais diversos contextos.
  Você deve ser capaz de identificar as nuncaes de comunicação do Ator alvo com coisas como:

  - se sempre inicia suas mensagens com determinados padrões
  - suas opiniões sobre os mais diversos assuntos e se mantem esse padrão em todos os contextos
  - como ele reage a diversas interações nos diferentes contextos de conversas

  Suas análises devem sempre ser conectadas a realidade das conversas que você teve acesso e no máximo pode fazer sugestões
  de como o ator poderia reagir em outros contextos, com base em seu padrão de mensagens mas sempre que fizer isso sinalize que está fazendo isso.
  """,
  verbose=True,
  allow_delegation=False,
  llm = llm,
  tools=[ readDirectoryTool ],
)

copycat_agent = Agent(
  role='Imitador de padrões de conversa',
  goal=f"""
  Dado um conjunto de mensagens de texto
  E dado um determinado Ator a ser copiado
  Sua expertise consiste em imitar os padrões de mensagens nos chats nos mais diversos contextos de conversa.
  Deve identificar e adotar o tom adequado de conversa, SEMPRE seguindo o padrão de conversas do Ator
  Leve em muita consideração o perfil traçado pelo Agente analista de perfil.
  """,
  backstory="""
  Você é um imitador profissional.
  Sua expertise consiste em identificar o contexto de uma mensagem e responder usando o padrão de mensagens do Ator copiado.
  Você deve ser capaz de identificar as nuncaes de comunicação do Ator e imitar com máxima perfeição.
  Suas respostas devem sempre ser conectadas a realidade das conversas que você teve acesso e no máximo pode fazer poucas modificações para dar maior fluidez e naturalidade a conversa.
  """,
  verbose=True,
  allow_delegation=False,
  llm = llm,
  tools=[ readDirectoryTool ],
)

def make_profile_analisy_task(target_actor, conversations_folder='/content/Conversas', assigned_agent=profiler_agent):
  return Task(
      agent=assigned_agent,
      description=f"""
      Conduza uma análise de todos os arquivos de texto na pasta de conversas.
      Identifique os pontos-chave dos padrões de conversação do {target_actor} e salve-os em seu contexto para uso futuro.
      """,
      expected_output = f"""
      Sua resposta final DEVE ser um relatório completo sobre o padrão de conversas do Ator contendo:
      - Os diferentes contextos em que {target_actor} se encontra
      - Exemplos de como {target_actor} inicia suas conversas nos diferentes contextos
      - Insight sobre a personalidade de {target_actor}
      - Faça uma cópia de sua análise em formato Markdown no seguinte arquivo de saída  /content/sample_data/profiler_analyst/{target_actor}_profiled.md, se o arquivo não existir, crie ele salve seu relatório Markdown nele.

      Sua resposta deve ser em Portugues do Brasil com um padrão de resposta formal e técnico.
      """
  )

def start_chat(target_actor, context='relação de amizade não tão próxima e com humor neutro', conversations_folder='/content/Conversas', assigned_agent=copycat_agent):
  return Task(
      agent=assigned_agent,
      description=f"""
      Deve iniciar uma convesa como {target_actor} usando os seus padrões de mensagem.
      Use para esse inicio de mensagem um padrão de contexto {context} ou mais próximo disso que conseguir.
      """,
      expected_output = f"""
      Sua resposta final DEVE ser uma unica mensagem de inicio de conversa como {target_actor} no contexto {context}.
      """
  )

def responds_chat(target_actor, message, conversations_folder='/content/Conversas', assigned_agent=copycat_agent):
  return Task(
      agent=assigned_agent,
      description=f"""
      Deve uma mensagem se passando por {target_actor} usando os seus padrões de mensagem.
      Se atente ao contexto de toda a conversa E
      Use um ton na resposta de acordo com a contexto ou mais próximo disso que conseguir.
      Se você receber como resposta alguma coisa que não puder comentar ou um texto ofensivo que não puder responder, use o padrão de mensagens de {target_actor} evitar a resposta
      Se você receber como resposta alguma contendo '\bye', use o padrão de mensagens de {target_actor} para se despedir.
      """,
      expected_output = f"""
      Sua resposta final DEVE ser uma unica mensagem de resposta a mensage {message}.
      """
  )



In [ ]:
# main.py

print("## Welcome to the Be Right Back Social Network")
print('----------------------------------------------')
print(f"** WARN: Before continue, please make sure you have already uploaded the conversation file **")
print("System ~: hangind to use: [pt-br]...")
print("System ~:Pronto, agora falamos em seu idioma nativo, vamos iniciar")

target_actor = input("System ~:- 1. Quem é o ator alvo da análise (nome que usa no whatsapp)?\n")
chats_folder = '/content/Conversas'
print("System ~: Ok! Passei as informações ai setor de análise de perfis")

crew_profiling = Crew(
    agents=[profiler_agent],
    tasks=[make_profile_analisy_task(target_actor)],
    verbose=2,
    process=Process.sequential
)
result = crew_profiling.kickoff()
print(result)


print("System~: Me preparando...")
crew_start_message = Crew(
    agents=[copycat_agent],
    tasks=[start_chat(target_actor)],
    verbose=0,
    process=Process.sequential
)

#TODO: Changes to responds in JSON
def crew_responds_message(actor, message):
  crew = Crew(
      agents=[copycat_agent],
      tasks=[responds_chat(target_actor, message)],
      verbose=0,
      process=Process.sequential
  )
  answer = crew.kickoff()
  return answer

start_message = crew_start_message.kickoff()
print("System~: Pronto, vamos lá! Quando desejar encerrar o chat digite: '\\bye'.")

received_message = input(f"@{target_actor} ~: ${start_message} \n")

#TODO: Improve it using generator function w/ yield instead of loop
while True:
  if '\bye' in received_message:
    model_answer = crew_responds_message(target_actor, received_message)
    print(model_answer)
    break

  model_answer = crew_responds_message(target_actor, received_message)
  received_message = input(f"@{target_actor} ~: ${model_answer}")

print("System~: Obrigado por utilizar o serviço da BRB, até a próxima")